In [ ]:
#REDEFINE to use easily on differnt computers, define to path this file is

#path for our github repositiory on local computer
rep_path='/Users/User1/Documents/GitHub/Rep2' 

#Check if path correct & working in this path
%cd $rep_path

In [ ]:
#select camera
cam_varr=0 #webcam
#cam_varr="rtmp://live.restream.io/live/re_6588559_f67e5f9e60fe5b825de6" #https://app.restream.io/

Package to install. Once per envrionment

In [ ]:
#dependancy of face recognition
%pip install CMake
%pip install dlib

In [ ]:
#installing open cv face recognition api
!pip3 install face_recognition

In [ ]:
#API form Model Garden for TensorFlow, needed for object detection API
#only need to run once
%cd $rep_path/WDS 
!git clone https://github.com/tensorflow/models/
%cd $rep_path/WDS/models/research

#proto buffer compilation, Protobufs used to conifure modele and trainning parapemetres
!protoc object_detection/protos/*.proto --python_out=.

#installing objeect dection api
%cp object_detection/packages/tf2/setup.py . #tf2 

!python -m pip install .

Imports. Every kerrnel

In [ ]:
#IMPORTING MODEL AND STARTING WEBCAM


import numpy as np
import time
import cv2  #real-time computer vision
import os
import face_recognition #libary for face rec

#to supperress tf logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'   
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
#supress tf logging
tf.get_logger().setLevel('ERROR')    

#for email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
#confirm cv2 version
import cv2
print(cv2.__version__)

In [ ]:
print(cv2.getBuildInformation())

Set up

In [ ]:
#learning face from sample image
#excpetion for when face couldnt be detected in sample image
try:
    gaurd01_pic = face_recognition.load_image_file("gaurd_01.jpg")
    gaurd01_encoding = face_recognition.face_encodings(gaurd01_pic)[0]
except IndexError:
    print("Couldn't find face in sample image")
    quit()

In [ ]:
#array of face encodings
known_gaurd_encodings = [
    gaurd01_encoding
]

#array of face labels
known_gaurd_label = [
    "gaurd01"
]

#intalize varribles
face_locations = []
face_encodings = []
face_label = []

#for analyzing every other frame for speed preformance (face)
analyze_this_frame = True

#skping frames for performance (weapon)
frames_per_cycle_w=2
frame_count_w=1 #offset from face

###defining paths of model##
label_path=rep_path+'/WDS/data/label_map.pbtxt'
configuration_path=rep_path+'/WDS/our_exported-models/our_model_2/pipeline.config'
checkpoint_path=rep_path+'/WDS/our_exported-models/our_model_2/checkpoint'

print(label_path)
print(configuration_path)
print(checkpoint_path)

####loading model####

#using GPU dynamic memory alloaton
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#builging dection model and loading pipleine configuration
configs = config_util.get_configs_from_pipeline_file(configuration_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

#restoring checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpoint_path, 'ckpt-0')).expect_partial()

#definig detection function too find wepaons in video
@tf.function
def detect_fn(image):

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

#for ploting loading labbel map dta
cat_index = label_map_util.create_category_index_from_labelmap(label_path,use_display_name=True)

#initial for messagee
last_email_sent_time = 0
current_time = time.time()

# Define the send_email function
def send_email(subject, body, from_email, to_email, password):
    # Create a MIMEMultipart message
    message = MIMEMultipart()
    message["From"] = from_email
    message["To"] = to_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Create SMTP session
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(from_email, password)

    # Send email
    text = message.as_string()
    server.sendmail(from_email, to_email, text)
    server.quit()

In [ ]:

####starting camera###

vid = cv2.VideoCapture(cam_varr) #video feed webcame

#geting frame size
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
size = (frame_width, frame_height)

#for saving video
#result = cv2.VideoWriter('WDSrecording.avi', cv2.VideoWriter_fourcc(*'MJPG'),10, size)

while(vid.isOpened()): #video capture is initizalized
    startTime=time.time()
    ret, frame=vid.read() #reads video frame: ret=bool if available frame, frame=image array
    frame_expanded = np.expand_dims(frame, axis=0)#match feed to deimisons of model
    
    ######DETECTING ELEMENTS##########
    ##########DETECTING FACE######

    #for analyzing every other frame for better speed preformance
    if analyze_this_frame:
        #to reducee frame size for speed perforomancee
        tiny_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # RGB(facee rec uses) <-- BGR(open cv)
        rgb_tiny_frame = cv2.cvtColor(tiny_frame, cv2.COLOR_BGR2RGB)
        
        # find locations of faces in frame
        face_locations = face_recognition.face_locations(rgb_tiny_frame)

        #getting encodings of located faces
        face_encodings = face_recognition.face_encodings(rgb_tiny_frame, face_locations)

        face_label = []
        for face_encoding in face_encodings:
            # comparing facee encodings found iin frame to our face encodings
            matches = face_recognition.compare_faces(known_gaurd_encodings, face_encoding)
            name = "Unknown"

            #assingingg to the most similar face
            face_distances = face_recognition.face_distance(known_gaurd_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_gaurd_label[best_match_index]
            face_label.append(name)

    #Alternatng frames for face
    analyze_this_frame = not analyze_this_frame
    ##########END DETECTING FACE######

    ##########DETECTING WEAPON######
    if (frame_count_w%frames_per_cycle_w)==0:
      in_tensor = tf.convert_to_tensor(np.expand_dims(frame, 0), dtype=tf.float32)
      detections, predictions_dict, shapes = detect_fn(in_tensor)
      label_id_offset = 1
    
    #to skip certian frames
    frame_count_w=frame_count_w+1
    ##########END DETECTING WEAPON######
    
    #######END DETECTING ELEMENTS##########

    # Showing results of face
    for (top, right, bottom, left), name in zip(face_locations, face_label):
        # with scale of orginal size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # create box 
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # labeling box
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        ##setting font
        font = cv2.FONT_HERSHEY_DUPLEX
        ##adding name
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Check if any weapons are detected
    if  current_time - last_email_sent_time > 300:
        # Check if it's been more than 5 minutes since the last email was sent
        if detections['detection_scores'][0].numpy().any()>=0.5:
            # Send an email alert
            subject = "Weapon detected!"
            body = "A weapon has been detected in the building. Please investigate."
            from_email = "seniordesignaud2023@gmail.com"
            to_email = "eng.abdullahmourad@gmail.com"
            password = "ctvlttuynqzxtmbb"
            send_email(subject, body, from_email, to_email, password)
            last_email_sent_time = current_time

    #dRAWING BOX OF DEDETED OBJECT(weaapon)
    viz_utils.visualize_boxes_and_labels_on_image_array(
        frame,
        detections['detection_boxes'][0].numpy(),
        (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
        detections['detection_scores'][0].numpy(),
        cat_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=300,
        min_score_thresh=.20,
        agnostic_mode=False)
    

    #Displaying footage to user
    if not ret:
         print("Exting. Didnt get frame")
         break
    else:
        #result.write(frame) #saving footage

        #display video with results, resize to orginal ratio
        cv2.imshow('WDS footage', cv2.resize(frame, size))

        #press q to quit 
        if cv2.waitKey(1) == ord('q'):
            break 

###ending webcam###
vid.release()
cv2.destroyAllWindows()
